In [ ]:
import pandas as pd
import os
import os.path
import chess
import chess.pgn

In [ ]:
def extractdata(pgn):
    '''
    input: pgn files of player
    output: 
        function returns step-by-step gameplay as a list
        function returns which player is playing White as a list
    
    list 'side' will be used to ensure only moves made by 
    intended player will be used in creation of the GAN
    '''
    
    side = []
    game_moves = []
    length = 100 #used for training purposes, to remove for DA servers
    for index in range(length):
        try:
            if chess.pgn.read_game(pgn).mainline_moves():
                # extracts game moves from the pgn files
                game_moves.append(chess.pgn.read_game(pgn).mainline_moves()) 
                
                # extracts player's name playing white from pgn files
                side.append(chess.pgn.read_game(pgn).headers["White"]) 
                
            print(index,chess.pgn.read_game(pgn))
            pass

    return game_moves, side

In [ ]:
def categorize_moves(game_moves, side, name):
    '''
    input: game_moves and side list from extractdata function
    output:
        function returns 2 lists, which contain all of player's move
        list X: player's moves when they are playing white
        list Y: player's moves when they are playing black 
    '''
    
    X = []
    Y = []
    match = 0
    
    for game in game_moves:
        board = chess.Board() # saves FEN notation of chess board
        white = side[match]
        if white == name:
            remainder = 0
        else:
            remainder = 1
        play = 0
        
        for move in game:
            if play % 2 == remainder: # creates list X of moves when the player is playing white
                X.append(board.copy())
            board.push(move) # move game forward one move
            if play % 2 == remainder: # creates list Y of moves when the player is playing black
                Y.append(board.copy())
            play = play + 1
        match = match + 1
    
    return X, Y

In [ ]:
adams_pgn = open("data/Adams.pgn")

def main_adams():
    game_moves, side = extractdata(adams_pgn)
    X, Y = categorize_moves(game_moves, side, 'Adams, Michael')

main_adams()